# Find lots of dams using WOFLs

**What does this notebook do?** This notebook uses output `NetCDF` files of WOFS summaries from `datacube-stats` to generate polygons of water bodies in the landscape. This code follows the following workflow:
* Generate a list of netCDF files within a specified folder location
* Opens each netCDF file and:
    * Keep only pixels identified as wet at least 10% of the time
    * Convert the raster data into polygons
    * Filter the polygons based on size and proximity to identified major rivers
    * Remove any named water bodies - e.g. lakes
* Append the final polygon set to a shapefile

**Required inputs:** 
This code requires that you have run `datacube-stats` to produce WOFS summaries as netCDF files. This code takes in the folder location to generate a list of netCDF files to process.

**Date:** July 2018

**Author:** Claire Krause

In [1]:
%pylab notebook

import rasterio.features
from shapely.geometry import Polygon, shape
import geopandas as gp
import fiona
from fiona.crs import from_epsg
import xarray as xr
import glob
import os.path

Populating the interactive namespace from numpy and matplotlib


## Set up the analysis

In [2]:
WOFSnetCDFFolder = '/g/data/r78/cek156/datacube_stats/WOFSDams/'

### Get the list of netcdf file names

In [3]:
Alltiles = glob.glob('{0}*.nc'.format(WOFSnetCDFFolder))

### Set up some file names for the inputs and outputs

In [4]:
# The name and filepath of the final output polygon set
WOFSshp = '/g/data/r78/cek156/dea-notebooks/Crop_mapping/Dams/AllNSW2013.shp'

# Read in the major rivers dataset
# We use the surface hydrology lines dataset(http://pid.geoscience.gov.au/dataset/ga/83107) to filter out 
# polygons that intersect with major rivers. 
# Note that we have filtered this dataset to only keep rivers tagged as `major`, and it is this filtered 
# dataset that we use here.
FilteredMajorRivers = '/g/data/r78/cek156/ShapeFiles/SurfaceHydrologyLinesRegionalFilteredMAJOR.shp'
MajorRivers = gp.GeoDataFrame.from_file(FilteredMajorRivers) 

## Read in the data

In [ ]:
for WOFSfile in Alltiles: 
    # Read in the data
    WOFSnetCDFData = xr.open_rasterio('NETCDF:{}:frequency'.format(WOFSfile))

    # Filter our classified data layer to remove noise
    # Remove any pixels that are wet < 10% of the time
    WOFSfiltered = WOFSnetCDFData > 0.25
    # Remove the superfluous time dimension
    WOFSfiltered = WOFSfiltered.squeeze()
    # Change all zeros to NaN
    WOFSfiltered = WOFSfiltered.where(WOFSfiltered !=0)

    # Convert the raster to polygons
    WOFSpolygons = rasterio.features.shapes(WOFSfiltered.data.astype('float32'), 
                                            transform = WOFSnetCDFData.transform[:-3])
    WOFSlist = list(WOFSpolygons)

    # Calculate the area of each polygon
    WOFLshapes = []
    for i, WOFLshape in enumerate(WOFSlist):
        polyArea = Polygon(WOFLshape[0]['coordinates'][0]).area
        WOFLshape[0]['properties'] = {'area': polyArea}
        WOFLshapes.append(WOFLshape)

    # Filter out any polygons smaller than 5000 m2 (i.e. 8 pixels)
    AreasIndex = [i for i, x in enumerate(WOFLshapes) if x[0]['properties']['area'] > 5050]
    WOFSOFSbig = [WOFLshapes[x] for x in AreasIndex]

    # Remove any polygons that intersect with a major river
    # We are only interested in farm dams, so do not need the WOFS polygons for the rivers. 
    WOFSOFSbreaktuple = [a for a, b in list(WOFSOFSbig)]

    # Grab the geometeries
    for poly in WOFSOFSbreaktuple:
        poly['geometry'] = shape(poly)

    WOFSOFSfiltered2p0 = gp.GeoDataFrame(WOFSOFSbreaktuple).set_geometry('geometry')
    Intersections= gp.sjoin(MajorRivers, WOFSOFSfiltered2p0, how="inner", op='intersects')
    IntersectIndex = sorted(list(set(Intersections['index_right'])))

    WOFSOFS = [WOFSOFSbreaktuple[x] for x in range(len(WOFSOFSbreaktuple)) 
                                      if x not in IntersectIndex]

    # Save the polygons to a shapefile
    schema = {'geometry': 'Polygon','properties': {'area': 'str'}}

    if os.path.isfile(WOFSshp):
        with fiona.open(WOFSshp, "a", crs = from_epsg(3577), driver = 'ESRI Shapefile', schema = schema) as output:
            for i in range(len(WOFSOFS)):
                output.write({'properties': {'area': WOFSOFS[i]['properties']['area']},
                            'geometry': {'type': WOFSOFS[i]['type'], 'coordinates': WOFSOFS[i]['coordinates']}})
    else:
        with fiona.open(WOFSshp, "w", crs = from_epsg(3577), driver = 'ESRI Shapefile', schema = schema) as output:
            for i in range(len(WOFSOFS)):
                output.write({'properties': {'area': WOFSOFS[i]['properties']['area']},
                            'geometry': {'type': WOFSOFS[i]['type'], 'coordinates': WOFSOFS[i]['coordinates']}})

/g/data/v10/public/modules/dea-env/20180629/lib/python3.6/site-packages/geopandas/tools/sjoin.py:53: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180629/lib/python3.6/site-packages/geopandas/tools/sjoin.py:53: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180629/lib/python3.6/site-packages/geopandas/tools/sjoin.py:53: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180629/lib/python3.6/site-packages/geopandas/tools/sjoin.py:53: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180629/lib/python3.6/site-packages/geopandas/tools/sjoin.py:53: UserWarning: CRS of frames being joined does not match!
  w

/g/data/v10/public/modules/dea-env/20180629/lib/python3.6/site-packages/geopandas/tools/sjoin.py:53: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180629/lib/python3.6/site-packages/geopandas/tools/sjoin.py:53: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180629/lib/python3.6/site-packages/geopandas/tools/sjoin.py:53: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180629/lib/python3.6/site-packages/geopandas/tools/sjoin.py:53: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180629/lib/python3.6/site-packages/geopandas/tools/sjoin.py:53: UserWarning: CRS of frames being joined does not match!
  w